In [298]:
import pandas as pd
import json
import random
import matplotlib.pyplot as plt
import datetime
from calendar import monthrange
import re
plt.style.use('dark_background')
pd.set_option('display.max_colwidth', None)

# Task 1 of 4

Given the files in then mox_technical_assessment folder:

Questions

## 1. Show us how you would structure them (and why). 
I start by opening them an inspecting their structure in json.
After inspection of the json file I guessed that is was a case of structure data with nested parameters.
I will start by flattening the json file which automatically joins all the keys by '\_' as column name.
Ending up with tabular data that can be stored in a pandas DataFrame. and easily exported to other applications

In [369]:
with open('mox_technical_assessment/customer-events.json', 'r') as f:
    ce = json.load(f) # Load raw jason data as a dictionary

with open('mox_technical_assessment/transaction-events.json', 'r') as f:
    te = json.load(f) # Load raw jason data as a dictionary

In [202]:
# Flatten customer jason records 
df_ce = pd.json_normalize(ce, sep='_')
# Flatten trx jason records
df_te = pd.json_normalize(te, sep='_')

## 2. How you would name the variables and tables (and why)
Looking closer at the structure I decide to change a bit the column names:
* remove payload since it is not adding information to the field. 
* "Detail" actually refers to _customer_ detail so I also change it to "customer"
* I also choose to change camel-case to snake-case: customerID -> customer_id

In [370]:
df_ce.columns = [c.replace('detail', 'customer').replace('payload_', '') for c in df_ce.columns]
df_ce = df_ce.rename(columns={'customerId': 'customer_id'})

df_te.columns = [c.replace('payload_', '') for c in df_te.columns]
df_te = df_te.rename(columns={'customerId': 'customer_id'})

I will now cast the types since many of the fields are stored as string but are actually int of float. 
I also check that the customer id is unique in the customer event table

In [371]:
# For customer set index to customer_id and assert there are no duplicates
# Cast age, salary to int and float
df_ce['customer_age'] = df_ce.customer_age.astype(int)
df_ce['customer_salary'] = df_ce.customer_salary.astype(float)
assert df_ce.index.duplicated().sum() == 0

In [204]:
# Parse the timestamp as such
df_te['timestamp'] = pd.to_datetime(df_te.timestamp)

A bit more complicated is the description field of the transaction events table. Upon inspection there seems to be a relatively easy way of obtaining the category of the event which can be very useful.

To extract it i use a regular expresion that will capture the last item in parenthesis in a line that does not include the character "(" in it. This is because some cases are like this: ```something (wan chai (rent)```
I looked at the resulting categories and it seems correct. However this is a risk in cases where the data changes since we don't know if the regex will work everytime. We would need to add checks no inform us of any new categories that might be a product of a mistake.

In [372]:
# Find last term in parenthesis in description, exclude matches that include a '('
df_te['category'] = df_te.description.apply(lambda s: re.findall('^.*\(([^\(]*?)\)$',s)[-1])

Finally since we have HKD and USD trx, we will need a table to convert, ideally we would call and API to obtain the exact conversion rate and the date of the event but since the HKD is pegged we can assume 7.75 without much problem

In [205]:
# For aggregates create a converted currency ammount column, I chose HKD but it could be USD too
df_curr = pd.DataFrame([['HKD', 1.], ['USD', 7.75]], columns=['transaction_currency', 'conv'])

In [ ]:

3. Download Tableau Desktop (Free Trial) from this link here, explore the data in detail using the
client
4. Walk us through what you’ve built, and tell us what type of patterns have you observed within
the data (including any anti-patterns or issues)

In [208]:
df_te.to_csv('files_for_tableau/transactions_events.csv', index=False)
df_ce.to_csv('files_for_tableau/customer_events.csv', index=False)
df_curr.to_csv('files_for_tableau/currency_exchange.csv', index=False)

## 3.Download Tableau Desktop (Free Trial) from this link here, explore the data in detail using the client

The tableau file is [here](./task1.twb)

## 4.Walk us through what you’ve built, and tell us what type of patterns have you observed within the data (including any anti-patterns or issues)
### 4.1 What did you consider when designing your data visualisations?
Tried to limit the number of visualizations to one dashboard. Top part assigned to transaction information and bottom plots to customer demographics. It is also interactive allowing other teams to investigate without needing to modify the dashboard or make it overly complicated.

As a summury of the main insights:
* Most of our transactions are in HKD, I've integrated the USD ammount at a rate of 7.75 HKD/USD. No mayor difference.
* The customers are relatively young. (average age of 32 years old) and not particulary skewed towards either female or male
* Since we have negative incomes I would assume that the income is self reported or it is calculated from the monthly balance change that did not come from payments. The median (reported?) income is 34K
* The top categories by total purchase amount are __travel agencies__ and __rent__ taking 31% and 27% repectively. They are followed by __other__, __Eating Places__, __Shopping__ and lastly with just 1.2% __Grocery__. 
* If we look at the Volume agains the Total purchase amount, we see that even though the Travel and rent are similar in total amount, rent has a much lower volume of activities (as one would expect). Similarly Eating places and shopping have are much more commong but do not contribute as much to the total amount.
* It is intersting to see that there is not seasonality for any of the categories throughout the month. or even when looking at the hour or day of the week. I would have expected Restaurants to rise during lunch/dinner hours or rent to peak at the end of the month. But we do not see this. Purchases happen 24/7.

### 4.2 Why did you choose this particular chart to tell the story?
I believe the area plot gives a good sense of the distribution along the categories of the payment amounts. The line and bar plot is a common plot and it is easy to relate to. It provides key information about trend and seasonality both for the total amounts and the volume of transactions. 

Age piramids are a widely used representation to understand the two key factors of the members demographics: age, gender. An income histogram is an easy way to understand the affluence of the customers which is the last piece of basic information not covered by the other visualizations

### 4.3 What could go wrong for users who use your chart to interpret the data? How would you improve it?
A key problem in this visualization is that the demographics are not per unique member. If a 30 year old engages 5 times more than a 60 year old then the plots will show this. A possibility is to generate a separate dashboard that only deals with the customer data looking at the member base regardless on whether they are purchasing or not. 

One of the things I would improve would be the number of statistics shown like totals, fractions means and such. On one hand this a good way to comunicate to other teams but it can also become a bit overwhelming, I would ask which KPI's or calculations are they confortable using and modify the dashboard to display them. 





# Task 2 of 4

Consider this recent study on VBs in Hong Kong from [Vpon.com](https://www.vpon.com/wp-content/uploads/The_New_Wave_of_Tech_Behemoths_in_Hong_Kong_EN.pdf?utm_source=VBHK_EN) and using the same dataset from
Task 1:

## 1. By utilising the demographic information on this report, how would you propose to use this data to optimise our acquisition funnel?

I would begin by comparing our member base to the one described in the report. For example:
* The gender distribution is completly different. Is is it beacause we are percieved differently? 
* Our data has a very large amount of trx on Hotels and travel agencies while the report mentions very little about it.

The purpose would be to give feedback to the team of the profile of our members compared to what the report considers to be "standard" to make sure that the people we are acquiring are in line with the company's strategy.

Secondly I would start to look at the descriptions of the transactions in detail. and See what seems to be trending, and check if there is any area we might be missing like Toys games or electronics. 

The main objective of this is to understand our member base in contrast to the "baseline" presented in the report to effectively personalize for the acquisition funnel. A very important part of this personalization would be a customer segmentaion or better still a persona analysis, a comprehensive definition of MOX's poster child customers.

## 2. How would you propose to segment the users?

It would depend on the exact case (is it for a specific campaing or strategy, is it a general persona, is it focused on customer value) and if would have to be done with other teams like customer insight team and strategy teams. But I would start by generating a "member model", a series of aggregations of features for each member. For example: 
* Demographic features: Age, gender, income
* Behavioral features:
    * Total money spend by category, frequency and recency of activity.
    * When is the money spend (based in the 01:00 to 04:00 activity seen in the report)
    * Try to massage a bit more the descriptions (NLP) to generate subcategories like the ones in the report: Games, Dating, Car etc.
    
I would then proceed to analyse the data with some dimensionality reduction technique like PCA or NMF and try to understand what are the main differences between our customers. I would keep in mind the report to guide the process to eliminate "noise" and weight appropiatly the information in this process, e.g. maybe the total amount ($) is not as important as the kind of purchase.

After this I would try several clustering approaches (KMeans or Hierarchical clustering to begin with). After a satisfactory clustering I would need to check with and insights team or strategy team to determine which ones are relevant which ones can be grouped toguether and add the qualitative side to the quantitative, so that the segmentation is align with any acquisition strategy in place.

# Task 3
Tasked is answered in the pdf task3.pdf

# TASK 4
I will create random example tables for the clase presented in the task.

We will create a df_prod table which contains the price and cost of a specific product (SKU). We will generate 100 SKU's

In [363]:
n_skus = 100 # number of products
prods = []
for i in range(n_skus): 
    sku = i + 100  # Start giving numbers from 100
    price = random.randint(20, 200)  # Each price will be a random number between 20 and 200 ($)
    cost = round(price * (0.7 + random.random()*0.29)) # The cost will be random between 70% and 99% of the price
    prods.append([sku, price, cost])
df_prod = pd.DataFrame(prods, columns=['sku', 'price', 'cost']).set_index('sku')

In [364]:
df_prod.head()

,price,cost
sku,,
100,70,59
101,65,52
102,159,138
103,60,50
104,31,23


We also need to create a sales table, it will contain what product was sold when

In [346]:
total_sales = 10000  # Lets create 10K sales
sales = []
for _ in range(total_sales):
    sku = df_prod.sample(1).index[0]  # Pick a random product
    year = random.randint(2019,2020)  # Randomly 2019, 2020
    month = random.randint(1,12)  # Any month
    day = random.randint(1, monthrange(year, month)[1])  # Any day of that month
    sales.append([sku, datetime.date(year, month, day)])  
df_sales = pd.DataFrame(sales, columns=['sku', 'date'])
df_sales['date'] = pd.to_datetime(df_sales['date'])  # Parse the date to datetime

Given this data we are now asked the following:
>A shopkeeper wants to know which combination of items he should stock in order to produce the
>highest profits. Produce a solution which presents the best selling products by volume and
>profitability, per day, week, month, year, and overall.

I am not being asked how to produce the highest profits, but to show the best selling products. The calculation procedure would be quite straight forward:

In [347]:
# Join sales with product information
df_join = df_sales.join(df_prod, on='sku', how='left')

In [348]:
# Calculate the profit from each sale and create convenient time features to filter and group by
df_join['profitability'] = df_join.price - df_join.cost 
df_join['year'] = df_join.date.dt.year
df_join['month'] = df_join.date.dt.month
df_join['week'] = df_join.date.dt.isocalendar().week
df_join['day'] = df_join.date.dt.day

In [366]:
# If we pivot on the desired time frames (e.g. year-month) and the product (sku) and then sum the profitability
# We will get a table that for each year-month has the profit generated by each product due to volume and product
# profitability
profit_table = pd.pivot_table(df_join,
               values='profitability',
               index=['year', 'month'], 
               columns=['sku'], 
               aggfunc = 'sum')
profit_table.head()

sku           100   101   102   103    104    105   106   107   108   109  \
year month                                                                  
2019 1      156.0  36.0  12.0  33.0  360.0   66.0  14.0  42.0  42.0  24.0   
     2      104.0  18.0  15.0  55.0  135.0   44.0  70.0  12.0  35.0  36.0   
     3       78.0  36.0  12.0  33.0  270.0  110.0  42.0  12.0  21.0  48.0   
     4       78.0  12.0  21.0  77.0  135.0   66.0  14.0  12.0  56.0  48.0   
     5      104.0  24.0  15.0  33.0  225.0  154.0  84.0  18.0  35.0  84.0   

sku         ...    190    191   192   193   194   195    196    197    198  \
year month  ...                                                              
2019 1      ...   26.0  148.0  60.0  64.0  10.0   6.0  230.0   20.0  175.0   
     2      ...   26.0  185.0  80.0  24.0   6.0   6.0    NaN  120.0  420.0   
     3      ...   78.0  185.0  40.0  16.0  10.0  12.0  276.0   80.0  140.0   
     4      ...  104.0   74.0  60.0  48.0   8.0   3.0  138.0   80.0  105.0   
     5      ...   26.0  111.0  60.0  32.0   6.0  24.0  230.0  120.0  140.0   

sku         199  
year month       
2019 1      3.0  
     2      4.0  
     3      3.0  
     4      4.0  
     5      3.0  

[5 rows x 100 columns]

In [368]:
# We now only need to capture the top k (e.g. 5) products for each time frame
# This can be done applying a lambda function to each row like below
profit_table.apply(lambda s: s.nlargest(5).index.tolist(), axis=1).head()

year  month
2019  1        [104, 183, 196, 182, 171]
      2        [198, 188, 161, 114, 132]
      3        [161, 196, 104, 115, 132]
      4        [132, 183, 120, 126, 171]
      5        [132, 171, 196, 104, 173]
dtype: object

So Jan 2019 product 104 was the most profitable, and may 2019 product 132.

This pipeline would have to be delivered either through a dashboard or an application.
Depending on how the shopkeper would like to recieve this information.
We would probably optimize the problem if the request is for an specific date so that we can save memory.